In [1]:
import pandas as pd
import requests
import os
import json
import math

In [17]:
df = pd.read_csv("./output/developerAddress.csv")
first_half = df[:4000]
other_half = df[4000:]
first_half

,DEVELOPER_ADDRESS
0,23 Stoneleigh Avenue\nBN1 8NP
1,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland"
2,"Perfect365, Inc.\n101 Jefferson Drive\nMenlo P..."
3,"4103, Limassol Cyprus"
4,"Gartengasse 8/8\n1050, Vienna\nAustria"
...,...
3995,438 St Vincent Street\nGlasgow\nG3 8EU\nUnited...
3996,Russia. Ekaterinburg. Kraulya 2
3997,"Jared B.\n5072 N. 300 W.\nProvo, Utah 84604"
3998,Level 9 Ariake Frontier Building Tower B\n3-7-...


In [3]:
df.shape

(8615, 1)

In [18]:
from decouple import config
API_KEY = config('API_KEY')

In [21]:
requests_fired = 0;
for i in range(4000, 4000+other_half.shape[0]):
    developer_addr = other_half.loc[i, "DEVELOPER_ADDRESS"]
    if (pd.isnull(developer_addr)):
        print("NaN")
        continue
    formmated = developer_addr.replace("#", " ")
    url = f"https://maps.googleapis.com/maps/api/place/autocomplete/json?input={formmated}&key={API_KEY}"
    payload={}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = json.loads(response.text)
    if not 'predictions' in data.keys():
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "NaN"
        continue
    if (data["predictions"] == []):
        other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = "undefined"
        continue
    prediction = data["predictions"][0]["description"]
    requests_fired +=1
    other_half.at[i, "DEVELOPER_ADDRESS"] = formmated
    other_half.at[i, "DEVELOPER_ADDRESS_PRED"] = prediction
other_half.head(20)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\Lenovo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
4000,"PO Box 12\nDewberry, AB\nT0B 1G0\nCanada",undefined
4001,"64 Mody Road, Tsim Sha Tsui East, Kowloon, Hon...","64 Mody Road, Tsim Sha Tsui East, Kowloon, Hon..."
4002,Vladimir Demirev\nSofia\nBulgaria,"Sofia, Bulgaria"
4003,"142 Henry St. 5W\nNew York, NY 10002","142 Henry St, New York, NY 10002, USA"
4004,"New York, USA","New York, NY, USA"
4005,"305, ST19, SEC1, AECHS, RWP, PK","Street 19, Sector 1 Airport Employees CHS, Raw..."
4006,Orrabacken 16\n827 52 Järvsö\nSweden,"Orrabacken 16, Järvsö, Sweden"
4007,"488 Topaz Lane\nBartlett, IL. 60103","488 Topaz Lane, Bartlett, IL 60103, USA"
4008,"10284 Canoe Brook Circle\nBoca Raton, FL\n33498","10284 Canoe Brook Circle, Boca Raton, FL 33498..."
4009,"Diego Palma 114,\nSan Isidro,\nBuenos Aires,\n...","Diego Palma 114, San Isidro, Buenos Aires, Arg..."


In [22]:
file_path = "./other_half.csv"
other_half.to_csv(path_or_buf=file_path, index=False);
print(requests_fired)

3883


In [23]:
first_half_df = pd.read_csv("./first_half.csv")
second_half_df = pd.read_csv("./other_half.csv")
combined_df = pd.concat([first_half_df, second_half_df])
combined_df.reset_index(drop=True, inplace=True)
combined_df

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
0,23 Stoneleigh Avenue\nBN1 8NP,"23 Stoneleigh Avenue, Brighton BN1 8NP, UK"
1,"ul. Łucka 2/4/6, lok. U 4,\n00-845 Warsaw\nPoland","Łucka, Warsaw, Poland"
2,"Perfect365, Inc.\n101 Jefferson Drive\nMenlo P...","101 Jefferson Drive, Menlo Park, CA 94025, USA"
3,"4103, Limassol Cyprus","Limassol Avenue 4103, Strovolos, Cyprus"
4,"Gartengasse 8/8\n1050, Vienna\nAustria","Gartengasse 8/8, 1050 Vienna, Austria"
...,...,...
8610,"501 W. Williams St. 480\nApex, NC 27502","501 W Williams St, Apex, NC 27502, USA"
8611,"4103\nImus City,\nCavite, PH","Philippine Veterans Bank - Imus, Emilio Aguina..."
8612,Calle de Cristóbal Bordiú 10. 1D.\n28003. Madr...,"Calle de Cristóbal Bordiú, 10, 28003 Madrid, S..."
8613,"9 rue Combe au Roucheret, 25770 Serre-les-Sapins","9 Rue Combe au Roucheret, 25770 Serre-les-Sapi..."


In [24]:
file_path = "./dev_addr.csv"
combined_df.to_csv(path_or_buf=file_path, index=False);

In [142]:
combined_df.head(10)

,DEVELOPER_ADDRESS,DEVELOPER_ADDRESS_PRED
0,23 Stoneleigh Avenue\nBN1 8NP,"23 Stoneleigh Avenue, Brighton BN1 8NP, UK"
1,"2500 Santa Clara Ave, Alameda, CA 94501","2500 Santa Clara Ave, Alameda, CA 94501, USA"
2,"Reading, Berkshire, RG6 3BT, United Kingdom","Sellafield Way, Lower Earley, Earley, Reading ..."
3,"2549-B Eastbluff Drive 201\nNewport Beach, CA...","2549 Eastbluff Drive, Newport Beach, CA 92660,..."
4,"POBOX 504905 DUBAI, UAE",Dubai - UAE
5,Güvenevler Mah.Keklik Cad.\nTürkiye,"Güvenevler, Cad Campus, Yenişehir/Mersin, Türkiye"
6,"12463 Rancho Bernardo Rd. 206, San Diego, CA ...","12463 Rancho Bernardo Road, San Diego, CA 9212..."
7,NaN,NaN
8,"4840 Hytech Drive\nCarson City, NV 89607","4840 Hytech Drive, Carson City, NV, USA"
9,"340/61 Quang Trung, Go Vap, Ho Chi Minh, Vietnam","340/61 Quang Trung, Gò Vấp, Ho Chi Minh City, ..."
